In [ ]:
import geopandas as gpd
import pandas as pd
import urllib.request as ur
import json
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import math
%matplotlib inline

In [ ]:
'''
RUNNING THE CODE REQUIRES 5 SIDEWALKOBSERVATIONS SHAPE FILES. 
https://data.seattle.gov/dataset/SidewalkObservations/q37p-ync7

CODE TAKES MORE THAN 30 MIN TO RUN IF >3 RADII
FOR TESTING, WOULD SUGGEST USING 1 OR 2 RADII

OUR CODE DID REFERENCE TO OTHER PEOPLE'S CODE. SEE THE INLINE COMMENT ON THE WEBSITE WE CONSULTED 
'''

'''
change the origin coordinates and radius below 
'''
origin_coord = [47.614412, -122.328409] 

# radius unit in km, in increasing order please
radius = [0.1, 0.2, 0.3, 0.4, 0.5]

'''
these are the 8 x and y coordinates representing N, NE, E, SE, ..., all of them 1 km away, to be scaled 

'''
sqrt2 = math.sqrt(2)
conversion_coord = [[1, 0], [1/sqrt2, -1/sqrt2], [0, -1], [-1/sqrt2, -1/sqrt2], [-1, 0], [-1/sqrt2, 1/sqrt2], [0, 1], [1/sqrt2, 1/sqrt2]]


'''
below commented code for optimizing (only select an area of points to compare), not yet implemented 
'''
# square = [[1,1], [1,-1], [-1,-1], [-1,1]]
# list_of_corners = []
# for k in range(len(square)):
#     x = origin_coord[0] + square[k][0] * radius[len(radius) - 1] * 180 / (math.pi * 6367)
#     y = origin_coord[1] + square[k][1] * radius[len(radius) - 1] * 180 / (math.pi * 6367 * math.cos(origin_coord[0]))
#     list_of_corners.append([x, y])
# list_of_corners_polygon = []
# list_of_corners_polygon.append([Polygon([(list_of_corners[0][0], list_of_corners[0][1]), (list_of_corners[1][0], list_of_corners[1][1]), (list_of_corners[2][0], list_of_corners[2][1]), (list_of_corners[3][0], list_of_corners[3][1])])])
# dataframe2 = pd.DataFrame(list_of_corners_polygon)
# dataframe2.columns = ['geometry']
# gdf2 = gpd.GeoDataFrame(dataframe2.geometry)

# all_data = gpd.read_file("SidewalkObservations.shp")
# all_data = all_data[all_data.SURFACE_CO != ""]
# issue_data = all_data

# for region in gdf2.geometry:
#     for point in all_data.geometry:
#         if point.within(region):
#             issue_data.append(point)

# print(issue_data)

'''
counts the number of issues for each point from the origin
'''
issues_output = []

# for all the radius
for i in range(0, len(radius)):
    # for all the directions
    for j in range(0, len(conversion_coord)):
        x = origin_coord[0] + conversion_coord[j][0] * radius[i] * 180 / (math.pi * 6367)
        y = origin_coord[1] + conversion_coord[j][1] * radius[i] * 180 / (math.pi * 6367 * math.cos(origin_coord[0]))
        destination_coord = [x, y]
   
        '''
        request this path to the coordinates caluclated from origin from api and convert to gpd format
        
        '''
        url = ur.urlopen('https://www.accessmap.io/api/v2/route.json?origin=' + str(origin_coord[0]) + '%2C' + str(origin_coord[1]) + '&destination=' + str(destination_coord[0]) + '%2C' + str(destination_coord[1]) + '')
        data = json.loads(url.read().decode())
        
        #https://stackoverflow.com/questions/21104592/json-to-pandas-dataframe
        coord = []
        coord.append([float(data['origin']['geometry']['coordinates'][0]), float(data['origin']['geometry']['coordinates'][1])])
        for result in data['routes'][0]['geometry']['coordinates']:
            coord.append(result)
        coord.append([float(data['destination']['geometry']['coordinates'][0]), float(data['destination']['geometry']['coordinates'][1])])

        df = pd.DataFrame(coord)
        
        #https://gis.stackexchange.com/questions/174159/convert-a-pandas-dataframe-to-a-geodataframe
        geometry = [Point(xy) for xy in zip(df[0], df[1])]
        gdf = gpd.GeoDataFrame(geometry=geometry)
        
        # https://gis.stackexchange.com/questions/95670/how-to-create-a-shapely-linestring-from-two-points
        list_of_buffered_path = []
        for k in range(0, len(gdf) - 1):
            list_of_buffered_path.append(LineString([gdf.geometry[k], gdf.geometry[k+1]]).buffer(0.0001))
        dataframe = pd.DataFrame(list_of_buffered_path)
        dataframe.columns = ['geometry']
        
        gdf = gpd.GeoDataFrame(dataframe.geometry)
        
        '''
        count number of issues
        
        '''
        #http://michelleful.github.io/code-blog/2015/04/29/geopandas-manipulation/
        
        all_data = gpd.read_file("SidewalkObservations.shp")
        all_data = all_data[all_data.SURFACE_CO != ""]

        count = 0
        for region in gdf.geometry:
            for point in all_data.geometry:
                if point.within(region) is True: 
                    count += 1
                    break
        
        issues_output.append([destination_coord[0], destination_coord[1], count])
        
print(issues_output)

In [ ]:
#http://geopandas.org/set_operations.html

'''
below code is for graphing given the number of issues at each coordinate
'''

# change the step size if needed
issues_count = [0, 5, 10, 15, 20, 25]
issues_count.sort(reverse=True)
#issues_count = 0;
polygons = []
numOfCoords = 0
polygon_list = []
#issues_output = [[47.6163717731903, -122.346023, 5], [47.61584463182746, -122.3445801257386, 3], [47.614572, -122.34398246765073, 0], [47.61329936817255, -122.3445801257386, 0], [47.6127722268097, -122.346023, 0], [47.61329936817255, -122.3474658742614, 5], [47.614572, -122.34806353234927, 1], [47.61584463182746, -122.3474658742614, 1], [47.618171546380594, -122.346023, 5], [47.61711726365491, -122.3431372514772, 6], [47.614572, -122.34194193530145, 3], [47.61202673634509, -122.3431372514772, 4], [47.61097245361941, -122.346023, 6], [47.61202673634509, -122.3489087485228, 6], [47.614572, -122.35010406469856, 4], [47.61711726365491, -122.3489087485228, 8], [47.619971319570894, -122.346023, 19], [47.61838989548237, -122.3416943772158, 12], [47.614572, -122.33990140295218, 6], [47.61075410451763, -122.3416943772158, 0], [47.60917268042911, -122.346023, 25], [47.61075410451763, -122.3503516227842, 23], [47.614572, -122.35214459704783, 4], [47.61838989548237, -122.3503516227842, 12], [47.621771092761186, -122.346023, 22], [47.61966252730983, -122.34025150295439, 10], [47.614572, -122.33786087060291, 9], [47.60948147269018, -122.34025150295439, 9], [47.60737290723882, -122.346023, 25], [47.60948147269018, -122.35179449704562, 23], [47.614572, -122.3541851293971, 22], [47.61966252730983, -122.35179449704562, 6], [47.623570865951486, -122.346023, 19], [47.62093515913728, -122.33880862869299, 11], [47.614572, -122.33582033825364, 7], [47.60820884086272, -122.33880862869299, 9], [47.60557313404852, -122.346023, 25], [47.60820884086272, -122.35323737130702, 23], [47.614572, -122.35622566174636, 23], [47.62093515913728, -122.35323737130702, 9]]
# change the graph colors if needed, include at least 1 color
colors = ['red', 'orange', 'yellow', 'green', 'blue', 'purple']

'''
A way of sorting the coordinates. Found that the other method below is better.
'''
# while issues_output is not None:
#     if len(polygon_list) > 2:
#         polygon_list = []
#     for issue in issues_output:
#         if issue[2] is issues_count:
#             polygon_list.append((issue[0], issue[1]))
#     issues_output = [x for x in issues_output if issue[2] is not issues_count]
#     if len(polygon_list) > 2 or len(issues_output) < 3:
#         if len(issues_output) < 3:
#             for issue in issues_output:
#                 polygon_list.append((issue[0], issue[1]))
#             issues_output = None
#             break
#         polygons.append(gpd.GeoSeries([Polygon(polygon_list)]))
#     issues_count += 1


'''
sort all the coordinates into different groups based on their issues
'''
for count in issues_count:
    # only make a new list if the length of last list >= 3
    if numOfCoords is 0:
        polygon_list = []
    for issue in issues_output:
        if issue[2] >= count:
            polygon_list.append((issue[0], issue[1]))
            numOfCoords += 1
    issues_output = [x for x in issues_output if not issue[2] >= count]
    # only make a new polygon if the number of coordinates >= 3
    if numOfCoords > 2 or len(issues_output) < 3:
        numOfCoords = 0
        if len(issues_output) < 3:
            for issue in issues_output:
                polygon_list.append((issue[0], issue[1]))
        print(polygon_list)
        polygons.append(gpd.GeoSeries([Polygon(polygon_list)]))
        if len(issues_output) < 3:
            break
print(polygons)

'''
graph out each group of polygons and stack them on top of each other
'''
#https://stackoverflow.com/questions/13872533/plot-different-dataframes-in-the-same-figure
gdf = gpd.GeoDataFrame(polygons[0])
gdf.columns = ['geometry']
ax = gdf.plot(color=colors[0])
for i in range(1, len(polygons)):
    gdf = gpd.GeoDataFrame(polygons[i])    
    gdf.columns = ['geometry']
    ax = gdf.plot(ax=ax, color=colors[i], alpha=0.3)
ax

